In [ ]:
from datasets import load_dataset

ds = load_dataset("wangrongsheng/ag_news")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
for i in range(10) :
    print(ds['train'][i])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'label': 2}
{'text': "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'label': 2}
{'text': 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'lab

In [ ]:
import re

def clean_text(text: str) -> str:
    """
    Basic text cleaning to prepare raw text for BERT tokenization / text classification.
    """
    if not isinstance(text, str):
        return ""
    # Lowercase
    text = text.lower()
    # Remove HTML tags (if any)
    text = re.sub(r"<.*?>", " ", text)
    # Remove non-ASCII / weird unicode (optional, depending on dataset)
    text = re.sub(r"[^\x00-\x7f]", " ", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # Remove emails
    text = re.sub(r"\S+@\S+", " ", text)
    # Replace any character that is not a letter, number or standard punctuation with space
    # (you may optionally allow punctuation)
    text = re.sub(r"[^a-z0-9\s.,!?\-']", " ", text)
    # Collapse multiple spaces into one
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
def apply_clean(example):
    example["text"] = clean_text(example["text"])
    return example

cleaned_dataset = ds.map(apply_clean)


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
cleaned_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
for i in range(3) :
  print('Original',ds['train']['text'][i])
  print('Cleaned',cleaned_dataset['train']['text'][i])
  print()
  print("-"*120)

Original Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Cleaned wall st. bears claw back into the black reuters reuters - short-sellers, wall street's dwindling band of ultra-cynics, are seeing green again.

------------------------------------------------------------------------------------------------------------------------
Original Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Cleaned carlyle looks toward commercial aerospace reuters reuters - private investment firm carlyle group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.

-----------------------

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [ ]:
# To load PyTorch weights and convert them to TensorFlow weights
model = TFAutoModel.from_pretrained("bert-base-uncased", from_pt=True)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Al

In [ ]:
# to vectorize the text
# prepare the inputs for bert as vectors
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(['Hello world', 'Hi how are you'], padding=True, truncation=True,
                  return_tensors='tf')
inputs

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.16888349,  0.13606296, -0.13940054, ..., -0.625113  ,
          0.05217265,  0.36714554],
        [-0.36327437,  0.14121908,  0.8799875 , ...,  0.10433027,
          0.28875798,  0.37267938],
        [-0.69859457, -0.698798  ,  0.06450206, ..., -0.2210371 ,
          0.00986854, -0.5939793 ],
        [ 0.8309833 ,  0.12366749, -0.15119022, ...,  0.10309616,
         -0.6779264 , -0.26285204],
        [-0.40266627, -0.01928267,  0.57325053, ..., -0.20656857,
          0.02338579,  0.20126319],
        [-0.6228407 , -0.2745353 ,  0.18117613, ..., -0.12944913,
         -0.03839113, -0.05733193]],

       [[ 0.09286542, -0.0263639 , -0.12239315, ..., -0.21063542,
          0.17386374,  0.17250949],
        [ 0.4074203 , -0.05930994,  0.5523467 , ..., -0.6790572 ,
          0.6555748 , -0.2945658 ],
        [-0.21155277, -0.685863  , -0.46280766, ...,  0.15278451

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

In [ ]:
emotions_encoded = cleaned_dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [ ]:
from collections import Counter

label_counts = Counter(cleaned_dataset["train"]["label"])
print(label_counts)


Counter({2: 30000, 3: 30000, 1: 30000, 0: 30000})


In [ ]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

emotions_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[  101,  1005,  3730, ...,     0,     0,     0],
       [  101,  1050,  1012, ...,     0,     0,     0],
       [  101,  3514, 22722, ...,     0,     0,     0],
       ...,
       [  101, 11260,  3138, ...,     0,     0,     0],
       [  101,  5619,  1005, ...,     0,     0,     0],
       [  101,  2085,  2489, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[0 0 2 2 2 2 2 2 2 3 3 1

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=4)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
1875/1875 [==============================] - 3017s 2s/step - loss: 0.2523 - accuracy: 0.9131
Epoch 2/3
1875/1875 [==============================] - 2949s 2s/step - loss: 0.1560 - accuracy: 0.9460
Epoch 3/3
1875/1875 [==============================] - 2947s 2s/step - loss: 0.1214 - accuracy: 0.9582


In [ ]:
classifier.evaluate(test_dataset)

119/119 [==============================] - 66s 530ms/step - loss: 0.1665 - accuracy: 0.9464


[0.1664750725030899, 0.9464473724365234]

In [ ]:
import os
import datetime

# Create a directory for saving models
save_dir = "./saved_models"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the entire model (architecture + weights + optimizer state)
model_save_path = os.path.join(save_dir, f"bert_sentiment_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}")
classifier.save(model_save_path, save_format='tf')

# Save the tokenizer separately
tokenizer_save_path = os.path.join(save_dir, "tokenizer")
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to: {model_save_path}")
print(f"Tokenizer saved to: {tokenizer_save_path}")

Model saved to: ./saved_models/bert_sentiment_20251206_162615
Tokenizer saved to: ./saved_models/tokenizer


In [ ]:
!zip -r model.zip /content/saved_models

  adding: content/saved_models/ (stored 0%)
  adding: content/saved_models/bert_sentiment_20251206_162615/ (stored 0%)
  adding: content/saved_models/bert_sentiment_20251206_162615/fingerprint.pb (stored 0%)
  adding: content/saved_models/bert_sentiment_20251206_162615/variables/ (stored 0%)
  adding: content/saved_models/bert_sentiment_20251206_162615/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/saved_models/bert_sentiment_20251206_162615/variables/variables.index (deflated 79%)
  adding: content/saved_models/bert_sentiment_20251206_162615/keras_metadata.pb (deflated 95%)
  adding: content/saved_models/bert_sentiment_20251206_162615/saved_model.pb (deflated 92%)
  adding: content/saved_models/bert_sentiment_20251206_162615/assets/ (stored 0%)
  adding: content/saved_models/tokenizer/ (stored 0%)
  adding: content/saved_models/tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/saved_models/tokenizer/tokenizer.json (deflated 71%)
  adding: con

In [ ]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>